# A General CTMC solver
EpiStochModels also provides a class to simulate any CTMC model you want, as long as you provide it with a transition matrix and propensity functions. So far this class is available for D programs, because of the impossibility of passing the propensity functions from Python into D. Nevertheless, We will demonstrate here how to use it:

Let's use the SIRD model as an example. The corresponding ODE model is:

\begin{align}
\frac{dS}{dt} &= -\beta S \frac{I}{N}\\
\frac{dI}{dt} &= \beta S \frac{I}{N} -\gamma I -\mu I\\
\frac{dR}{dt} &= \gamma I\\
\frac{dD}{dt} &= \mu I
\end{align}

For this model, the following events are possible:


1. **Infection:** propensity: $\beta S \frac{I}{N}$
1. **Recovery:** propensity: $\gamma I$
1. **Death:** propensity: $\mu I$

The following transition matrix describe the changes in the state of the system when these events happen:

TM = \begin{bmatrix}
-1 & 1 & 0 & 0\\
0 & -1 & 1 & 0\\
0 & -1 & 0 & 1
\end{bmatrix}.

The D code required to implement this model is:

```D
int[][] tmat = [[-1,1,0,0],
                    [0,-1,1,0], 
                    [0,-1,0,1]];
double[string] pars = [
    "beta": 0.3,
    "gam": 0.05,
    "mu": 0.01
];
alias double delegate(int[], double[string]) dy;
dy[] props = [
    (v,p)=> p["beta"]*v[0]*v[1]/sum(v[0..$-1]), // infection
    (v,p)=> p["gam"]*v[1], // recovery
    (v,p)=> p["mu"]*v[1] // death
    ];
CTMC model = new CTMC(tmat, props);
model.initialize([995,5,0,0],pars);
auto res = model.run(0, 1000);
```